In [22]:
import pandas as pd
import numpy as np

In [19]:
galaxies_full = pd.read_csv("galaxydatabase.csv", encoding = "ISO-8859-1")
renamed = galaxies_full.rename(columns={galaxies_full.columns[3]: "Distance"})
galaxies = renamed[["Name", "Distance", "RadRa", "RadDec"]]
galaxies["Distance"][0] = 9811.0
galaxies

C:\Users\hanna\AppData\Local\Temp\ipykernel_25800\3066824220.py:4: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  galaxies["Distance"][0] = 9811.0
C:\Users\hanna\AppData\Local\Temp\ipykernel_25800\3066824220.py:4: SettingWithCopyWarning: 
A v

,Name,Distance,RadRa,RadDec
0,NGC 628,9811.0,0.418879,0.275471
1,NGC 477,101000.0,0.353429,0.706567
2,NGC 364,101900.0,0.279253,-0.013963
3,NGC 64,102000.0,0.074176,-0.118973
4,NGC 90,102350.0,0.091630,0.390954
...,...,...,...,...
389,NGC 934,96300.0,0.641409,-0.004072
390,NGC 409,96400.0,0.301069,-0.624828
391,NGC 2,96800.0,0.030543,0.482874
392,NGC 223,97700.0,0.183260,0.014544


In [21]:
transposed = galaxies.transpose()
transposed

,0,1,2,3,4,5,6,7,8,9,...,384,385,386,387,388,389,390,391,392,393
Name,NGC 628,NGC 477,NGC 364,NGC 64,NGC 90,NGC 949,NGC 850,NGC 631,NGC 102,NGC 334,...,NGC 325,NGC 66,NGC 799,NGC 653,NGC 431,NGC 934,NGC 409,NGC 2,NGC 223,NGC 219
Distance,9811.0,101000.0,101900.0,102000.0,102350.0,10300.0,103000.0,103850.0,104000.0,104000.0,...,94550.0,94600.0,95000.0,96000.0,96150.0,96300.0,96400.0,96800.0,97700.0,98300.0
RadRa,0.418879,0.353429,0.279253,0.074176,0.09163,0.654498,0.571595,0.418879,0.10472,0.253073,...,0.248709,0.082903,0.532325,0.445059,0.322886,0.641409,0.301069,0.030543,0.18326,0.18326
RadDec,0.275471,0.706567,-0.013963,-0.118973,0.390954,0.648099,-0.025889,0.101811,-0.243473,-0.612611,...,-0.089012,-0.400262,-0.001745,0.621919,0.588176,-0.004072,-0.624828,0.482874,0.014544,0.015708


In [29]:
rad = np.pi/2
g1RA = galaxies.loc[0, "RadRa"]
g2RA = transposed.loc["RadRa", 1]
g1DC = galaxies.loc[0, "RadDec"]
g2DC = transposed.loc["RadDec", 1]

cosofgamma = np.cos(rad - g1DC)*np.cos(rad - g2DC) + np.sin(rad - g1DC)*np.sin(rad - g2DC)*np.cos(g1RA - g2RA)
cosofgamma

0.9069410844937071

In [37]:
def cos_of_gamma():
    gammas = np.array([])
    rad = np.pi/2
    for i in range(len(galaxies)):
        g1RA = galaxies.loc[i, "RadRa"]
        g1DC = galaxies.loc[i, "RadDec"]
        for j in range(len(transposed)):
            g2RA = transposed.loc["RadRa", j]
            g2DC = transposed.loc["RadDec", j]

            one_cos_gamma = np.cos(rad - g1DC)*np.cos(rad - g2DC) + np.sin(rad - g1DC)*np.sin(rad - g2DC)*np.cos(g1RA - g2RA)
            gammas = np.append(gammas, one_cos_gamma)
    return gammas
    
        


In [39]:
gammas = np.arccos(cos_of_gamma())


C:\Users\hanna\AppData\Local\Temp\ipykernel_25800\2236797763.py:1: RuntimeWarning: invalid value encountered in arccos
  gammas = np.arccos(cos_of_gamma())
